In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , BatchNormalization , Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import log_loss, roc_auc_score , accuracy_score

In [ ]:
df_1458=pd.read_csv('/kaggle/input/processed/processed_dataset/df_1458.csv')
df_1458.head()

In [ ]:
X = df_1458
target_click = pd.read_csv('/kaggle/input/processed/processed_dataset/is_clk_1458.csv')
target_click.head()

In [ ]:
target_click['is_clk'].value_counts()

In [ ]:
X_train1, X_temp, y_train1_click, y_temp = train_test_split(X, target_click, test_size=0.1, random_state=42)
X_test,X_valid,y_test,y_valid = train_test_split(X_temp,y_temp,test_size=0.5,random_state=42)
smote = SMOTE(sampling_strategy=0.4, random_state=42)
X_train, y_train_click = smote.fit_resample(X_train1, y_train1_click)

print(X_train.shape)
print(y_train_click.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'), 
    Dropout(0.3),  
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.3), 
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') 
])

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,          
    restore_best_weights=True,  
    verbose=1
)

In [ ]:
history = model.fit(X_train, y_train_click,
                    validation_data=(X_valid, y_valid),
                    epochs=50,
                    batch_size=128,
                    verbose=1,
                    callbacks=[early_stopping])

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_binary = (y_pred > 0.99).astype(int)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_binary)
roc_auc = auc(fpr, tpr)

In [ ]:
from sklearn.metrics import log_loss
logloss = log_loss(y_test, y_pred)
print("Log Loss:", logloss)
print(f'Test ROC AUC: {roc_auc}')

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='black', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='upper left')
plt.text(tpr[-1] - 0.4, fpr[1], "Advertiser id:1458 (Click)", fontsize=12, color='black')
plt.savefig('/kaggle/working/clk_1458.jpg')
plt.show()

In [ ]:
model.save('clk_1458.h5')
model.save_weights('clk_1458.weights.h5')